In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xvf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark
!pip install -q pyspark

import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import matplotlib.pyplot as plt


os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "D:/spark-3.5.0-bin-hadoop3"  # Измените на ваш путь установки Spark

import findspark
findspark.init()

spark = SparkSession.builder.master("local[*]").appName("Анализ данных Iris").getOrCreate()


df = spark.read.csv('D:/iris.csv', header=True, inferSchema=True)  # Измените на ваш путь к файлу


df.groupBy("species").agg(
    F.max("sepal_length"), F.min("sepal_length"), F.avg("sepal_length"),
    F.max("sepal_width"), F.min("sepal_width"), F.avg("sepal_width"),
    F.max("petal_length"), F.min("petal_length"), F.avg("petal_length"),
    F.max("petal_width"), F.min("petal_width"), F.avg("petal_width")
).show()


pd_df = df.toPandas()


plt.scatter(pd_df['sepal_length'], pd_df['sepal_width'], c=pd_df['species_encoded'])
plt.xlabel('Длина чашелистика')
plt.ylabel('Ширина чашелистика')
plt.title('Размеры чашелистиков')
plt.show()


plt.scatter(pd_df['petal_length'], pd_df['petal_width'], c=pd_df['species_encoded'])
plt.xlabel('Длина лепестка')
plt.ylabel('Ширина лепестка')
plt.title('Размеры лепестков')
plt.show()


threshold = 2.0  
df = df.withColumn("predicted_species", F.when(df["petal_length"] < threshold, "Setosa").otherwise("Unknown"))


correct_predictions = df.filter((df["predicted_species"] == "Setosa") & (df["species"] == "Setosa")).count()
incorrect_predictions = df.filter((df["predicted_species"] == "Setosa") & (df["species"] != "Setosa")).count()

print(f"Правильные предсказания: {correct_predictions}")
print(f"Неправильные предсказания: {incorrect_predictions}")